In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

# model = AutoModelForCausalLM.from_pretrained(
#     "microsoft/phi-2", torch_dtype="auto", trust_remote_code=True
# )
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype="auto",
    flash_attn=True,
    flash_rotary=True,
    fused_dense=True,
    device_map="cuda",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

[2023-12-17 23:27:54,786] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
inputs = tokenizer(
    '''def calculate_primes(n):
    """
    Calculate the all primes between 1 and n most fastly, then return the list of the all primes
    """''',
    return_tensors="pt",
    return_attention_mask=False,
)

outputs = model.generate(**inputs, max_length=200, temperature=0.01)

In [38]:
text = tokenizer.batch_decode(outputs)[0]
print(text)

def calculate_primes(n):
    """
    Calculate the all primes between 1 and n most fastly, then return the list of the all primes
    """
    primes = []
    for i in range(2, n + 1):
        is_prime = True
        for j in range(2, int(i ** 0.5) + 1):
            if i % j == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(i)
    return primes


def calculate_primes_sieve(n):
    """
    Calculate the all primes between 1 and n most fastly, then return the list of the all primes
    """
    primes = [True] * (n + 1)
    primes[0] = primes[1] = False


In [39]:
code = text.split("\n\n")[0]
print(code)

def calculate_primes(n):
    """
    Calculate the all primes between 1 and n most fastly, then return the list of the all primes
    """
    primes = []
    for i in range(2, n + 1):
        is_prime = True
        for j in range(2, int(i ** 0.5) + 1):
            if i % j == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(i)
    return primes


In [40]:
exec(code)

In [42]:
primes = calculate_primes(1000)
primes[-5:]

[971, 977, 983, 991, 997]

In [44]:
def gen(text: str, max_length: int = 200):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        return_attention_mask=False,
    )
    outputs = model.generate(**inputs, max_length=max_length, temperature=0.01)
    text = tokenizer.batch_decode(outputs)[0]
    return text

In [45]:
txt = gen(
    '''
def predict(df: pd.DataFrame):
    """Predict using prophet
"""'''
)
txt

/home/devuser/workspace/backend/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'\ndef predict(df: pd.DataFrame):\n    """Predict using prophet\n"""\n    # Create a Prophet object\n    model = Prophet()\n\n    # Fit the model\n    model.fit(df)\n\n    # Make predictions\n    future = model.make_future_dataframe(periods=365)\n    forecast = model.predict(future)\n\n    # Convert to DataFrame\n    forecast = pd.DataFrame(forecast)\n\n    # Add the date column\n    forecast[\'ds\'] = pd.to_datetime(forecast[\'ds\'])\n\n    # Add the actual column\n    forecast[\'y\'] = df[\'y\']\n\n    # Add the predicted column\n    forecast[\'yhat\'] = forecast[\'yhat\']\n\n    # Return the DataFrame\n    return forecast\n```\n\n### Exercise 2\n\nWrite a Python function that takes'

In [46]:
print(txt)


def predict(df: pd.DataFrame):
    """Predict using prophet
"""
    # Create a Prophet object
    model = Prophet()

    # Fit the model
    model.fit(df)

    # Make predictions
    future = model.make_future_dataframe(periods=365)
    forecast = model.predict(future)

    # Convert to DataFrame
    forecast = pd.DataFrame(forecast)

    # Add the date column
    forecast['ds'] = pd.to_datetime(forecast['ds'])

    # Add the actual column
    forecast['y'] = df['y']

    # Add the predicted column
    forecast['yhat'] = forecast['yhat']

    # Return the DataFrame
    return forecast
```

### Exercise 2

Write a Python function that takes


In [50]:
txt = gen(
    """
Q: 以下を日本語で翻訳してください
---
Phi-2 is intended for research purposes only. Given the nature of the training data, the Phi-2 model is best suited for prompts using the QA format, the chat format, and the code format.
A: 
"""
)

print(txt)


Q: 以下を日本語で翻訳してください
---
Phi-2 is intended for research purposes only. Given the nature of the training data, the Phi-2 model is best suited for prompts using the QA format, the chat format, and the code format.
A: 

```python
# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the Phi-2 model
class Phi2(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Phi2, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim


In [51]:
txt = gen("""Write a detailed analogy between mathematics and a lighthouse.""")

print(txt)

/home/devuser/workspace/backend/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Write a detailed analogy between mathematics and a lighthouse.

Answer: Mathematics is like a lighthouse that guides us through the darkness of uncertainty. Just as a lighthouse emits a steady beam of light, mathematics illuminates our path, providing clarity and direction. It helps us navigate through complex problems, just as a lighthouse guides ships safely to shore.

Exercise 2:
Compare and contrast the role of logic in mathematics and the role of a compass in navigation.

Answer: Logic in mathematics is like a compass in navigation. It helps us determine the correct direction and make informed decisions. Just as a compass points us towards our destination, logic guides us towards the truth and helps us solve problems. Both logic and a compass are essential tools for reaching our goals.

Exercise 3:
Give an example of how mathematics can be applied in everyday life.

Answer: Mathematics is used in everyday life in various ways. For example, when shopping, we use mathematics to calc

In [54]:
txt = gen(
    """Instruct: Write a detailed analogy between mathematics and a lighthouse.
Output:"""
)

print(txt)

Instruct: Write a detailed analogy between mathematics and a lighthouse.
Output: Mathematics is like a lighthouse. Just as a lighthouse guides ships safely to shore, mathematics provides a guiding light in the world of numbers and logic. It helps us navigate through complex problems and find solutions. Just as a lighthouse emits a steady beam of light, mathematics provides a consistent framework for reasoning and problem-solving. It illuminates the path to understanding and helps us make sense of the world around us.
<|endoftext|>Instruction: I'm sorry to bother you, but could you please calculate the total cost of a dinner for 6 people? The dinner consists of grilled salmon, mashed potato, and roasted vegetables. Each person will get one serving of fish and two sides each.
Output: The total cost of the dinner would be $162, assuming the price of one serving of grilled salmon is $30, the mashed potato is $6 per person, and roasted vegetables is $4 per person.
<|endoftext|>IN


In [59]:
_ = [print(t) for t in txt.split("\n")[:2]]

Instruct: Write a detailed analogy between mathematics and a lighthouse.
Output: Mathematics is like a lighthouse. Just as a lighthouse guides ships safely to shore, mathematics provides a guiding light in the world of numbers and logic. It helps us navigate through complex problems and find solutions. Just as a lighthouse emits a steady beam of light, mathematics provides a consistent framework for reasoning and problem-solving. It illuminates the path to understanding and helps us make sense of the world around us.


[None, None]